# Jupyter notebook sample

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

In [2]:
import sys
print(sys.executable)

/Users/sofiascaiano/Documents/maestria/Data Mining EyF/churn/venv/bin/python


In [16]:
%%sql
--create or replace table base as
select
    * REPLACE(last_day(make_date(
    CAST(SUBSTR(CAST(foto_mes AS VARCHAR), 1, 4) AS INTEGER),
    CAST(SUBSTR(CAST(foto_mes AS VARCHAR), 5, 2) AS INTEGER),
    1
  )) as foto_mes)
from read_csv_auto("/Users/sofiascaiano/Documents/maestria/Data Mining EyF/datasets/competencia_01_crudo.csv")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221323,2021-01-31,1,0,0,46,93,3070.24,15691.10,1278.11,...,0.0,4,0.00,-20833.47,0.0,2814,6743.38,6,0,9559.95
1,249227600,2021-01-31,1,0,0,42,278,496.21,15321.80,418.97,...,0.0,11,13267.49,-9662.51,0.0,5541,9419.90,8,0,1900.26
2,249234235,2021-01-31,1,0,0,54,17,806.56,10549.80,174.26,...,0.0,4,0.00,-4692.00,0.0,577,2821.32,2,0,6885.51
3,249244449,2021-01-31,1,0,0,61,290,27485.71,65621.74,23590.68,...,0.0,25,23800.17,-20349.22,0.0,7916,23509.71,8,0,1219.92
4,249244739,2021-01-31,1,0,0,66,320,3306.82,48787.99,3274.95,...,0.0,4,0.00,-88178.37,0.0,1958,20397.54,8,0,2627.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,1598368433,2021-06-30,1,0,0,58,1,4123.09,4123.09,2242.51,...,0.0,7,0.00,0.00,0.0,28,12521.76,2,0,1747.77
978435,1598419415,2021-06-30,1,0,0,30,1,-2.26,-2.26,0.00,...,NaN,7,0.00,NaN,NaN,9,NaN,<NA>,<NA>,0.00
978436,1598425905,2021-06-30,1,0,0,28,1,5.92,5.92,0.00,...,NaN,0,0.00,NaN,NaN,20,NaN,<NA>,<NA>,0.00
978437,1598470389,2021-06-30,1,0,0,78,1,244.96,244.96,0.00,...,NaN,7,0.00,NaN,NaN,15,NaN,<NA>,<NA>,0.00


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978439 entries, 0 to 978438
Columns: 154 entries, numero_de_cliente to Visa_mpagominimo
dtypes: Int64(17), datetime64[us](1), float64(72), int64(64)
memory usage: 1.1 GB


In [23]:
%%sql
-- version original
select t0.numero_de_cliente, 
       t0.foto_mes, 
       case when t1.foto_mes is null then 'BAJA+1' when t2.foto_mes is null then 'BAJA+2' else 'CONTINUA' end as target
from df as t0
left join df as t1
on t0.numero_de_cliente = t1.numero_de_cliente
and last_day(date_add(t0.foto_mes, INTERVAL 1 MONTH)) = t1.foto_mes
left join df as t2
on t0.numero_de_cliente = t2.numero_de_cliente
and last_day(date_add(t0.foto_mes, INTERVAL 2 MONTH)) = t2.foto_mes
where t0.foto_mes <= '2021-04-30'


,numero_de_cliente,foto_mes,target
0,1213852109,2021-01-31,CONTINUA
1,1213858300,2021-01-31,CONTINUA
2,1213872818,2021-01-31,CONTINUA
3,1213876242,2021-01-31,CONTINUA
4,1213881716,2021-01-31,CONTINUA
...,...,...,...
650204,567362947,2021-01-31,BAJA+1
650205,830565182,2021-01-31,BAJA+1
650206,931285617,2021-04-30,BAJA+1
650207,1184545201,2021-01-31,BAJA+1


In [24]:
# df_target.groupby("foto_mes")['target'].value_counts()

foto_mes    target  
2021-01-31  CONTINUA    160080
            BAJA+2         825
            BAJA+1         622
2021-02-28  CONTINUA    160292
            BAJA+2        1032
            BAJA+1         831
2021-03-31  CONTINUA    161119
            BAJA+1        1039
            BAJA+2         951
2021-04-30  CONTINUA    161333
            BAJA+2        1130
            BAJA+1         955
Name: count, dtype: int64

In [25]:
%%sql
-- version sin considerar como baja a aquellos clientes que luego vuelven a aparecer con antiguedad
with cte as (
select t0.numero_de_cliente, 
       t0.foto_mes, 
       max(t1.foto_mes) max_mes_total,
       min(t1.cliente_antiguedad) as min_antiguedad,
       max(t2.foto_mes) as max_mes
from df as t0
left join df as t1
on t0.numero_de_cliente = t1.numero_de_cliente
and t0.foto_mes < t1.foto_mes
left join df as t2
on t0.numero_de_cliente = t2.numero_de_cliente
and t2.foto_mes > t0.foto_mes
and t2.foto_mes <= last_day(date_add(t0.foto_mes, INTERVAL 2 MONTH))
group by all
)

select numero_de_cliente, 
       foto_mes, 
       min_antiguedad,
       case when max_mes_total is null then 'BAJA+1' 
            when max_mes_total = last_day(date_add(foto_mes, INTERVAL 1 MONTH)) then 'BAJA+2' 
            else 'CONTINUA' 
       end as target,
       max_mes
from cte
where foto_mes < (select max(foto_mes) - INTERVAL 1 MONTH from cte)

,numero_de_cliente,foto_mes,min_antiguedad,target,max_mes
0,556999279,2021-01-31,89,CONTINUA,2021-03-31
1,558453084,2021-01-31,39,CONTINUA,2021-03-31
2,560554986,2021-01-31,34,CONTINUA,2021-03-31
3,550273948,2021-01-31,217,CONTINUA,2021-03-31
4,550297356,2021-01-31,127,CONTINUA,2021-03-31
...,...,...,...,...,...
650204,670796345,2021-01-31,<NA>,BAJA+1,NaT
650205,277977486,2021-03-31,<NA>,BAJA+1,NaT
650206,453387495,2021-03-31,<NA>,BAJA+1,NaT
650207,481060111,2021-03-31,<NA>,BAJA+1,NaT


In [32]:
%%sql
SELECT base.*, df_target.target
FROM base 
LEFT JOIN df_target
ON base.numero_de_cliente = df_target.numero_de_cliente
AND base.foto_mes = df_target.foto_mes

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,target
0,675532669,2021-04-30,1,0,0,50,178,1193.57,15706.44,1752.49,...,2,0.00,-7458.02,0.0,4768,882.97,1,0,2076.21,CONTINUA
1,675533540,2021-04-30,1,0,0,37,53,4001.90,41903.32,396.72,...,2,0.00,-12046.71,0.0,1356,21609.26,12,0,23800.17,CONTINUA
2,675538066,2021-04-30,1,0,0,60,67,1800.26,11617.46,2604.38,...,2,0.00,-19458.82,0.0,2034,6452.63,8,0,1524.90,CONTINUA
3,675541858,2021-04-30,1,0,0,38,144,-500.28,6642.40,781.60,...,2,0.00,-107935.94,0.0,4365,85692.09,14,0,5982.30,CONTINUA
4,675552217,2021-04-30,1,0,0,33,178,41.96,-4.16,216.18,...,16,11485.59,-12493.45,0.0,3267,1689.17,3,0,2087.94,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,1598368433,2021-06-30,1,0,0,58,1,4123.09,4123.09,2242.51,...,7,0.00,0.00,0.0,28,12521.76,2,0,1747.77,None
978435,1598419415,2021-06-30,1,0,0,30,1,-2.26,-2.26,0.00,...,7,0.00,NaN,NaN,9,NaN,<NA>,<NA>,0.00,None
978436,1598425905,2021-06-30,1,0,0,28,1,5.92,5.92,0.00,...,0,0.00,NaN,NaN,20,NaN,<NA>,<NA>,0.00,None
978437,1598470389,2021-06-30,1,0,0,78,1,244.96,244.96,0.00,...,7,0.00,NaN,NaN,15,NaN,<NA>,<NA>,0.00,None


In [28]:
len(df_merge)

978439

In [33]:
df_merge['target'].value_counts(dropna=False)

target
CONTINUA    642844
None        328230
BAJA+2        3931
BAJA+1        3434
Name: count, dtype: int64

In [31]:
df_merge[df_merge['target'].isna()]['foto_mes'].unique()

<DatetimeArray>
['2021-05-31 00:00:00', '2021-06-30 00:00:00']
Length: 2, dtype: datetime64[us]

In [35]:
df_merge.to_csv("/Users/sofiascaiano/Documents/maestria/Data Mining EyF/datasets/competencia_01_target.csv", index=False)